In [2]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [3]:
train_df = pd.read_csv('input/train.csv')
# train_df = pd.read_csv('input/train_min.csv')
test_df = pd.read_csv('input/test.csv')

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [35]:
# parameter tuning using grid search

params = {
         'objective': ['binary'],
         'max_depth': [127],
         'metric': ['auc'],
         'boosting':['gbdt'],
         }

lbgm = lgb.LGBMClassifier(objective='binary')
grid = GridSearchCV(estimator=lbgm, param_grid=params, cv=5)
grid.fit(X, y)

print(grid.best_params_)
print(grid.best_score_)

{'boosting': 'gbdt', 'max_depth': 127, 'metric': 'auc', 'objective': 'binary'}
0.90775


In [39]:
n_splits = 2
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [40]:
'''
# cv: 0.89951
params = {
         'num_leaves': 8,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4}
'''

# cv: 0.90775
# lb: 0.891 with fold_n=2 => overfit?
params = {
         'objective': 'binary',
         'max_depth': 127,
         'metric': 'auc',
         'boosting':'gbdt',
         }

In [41]:
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=20000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
    
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/n_splits

Fold 0 started at Fri Mar  1 11:06:22 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.988736	valid_1's auc: 0.885096
Early stopping, best iteration is:
[369]	training's auc: 0.99398	valid_1's auc: 0.885953
Fold 1 started at Fri Mar  1 11:07:13 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.989106	valid_1's auc: 0.881934
[600]	training's auc: 0.999521	valid_1's auc: 0.882709
Early stopping, best iteration is:
[571]	training's auc: 0.999308	valid_1's auc: 0.882859


In [42]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission.csv", index=False)